<a href="https://colab.research.google.com/github/comp-aspects-of-appl-linguistics/relation-extraction-utils/blob/master/results/ucca_precision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# download ucca/tupa modules

!git clone https://github.com/danielhers/ucca.git
!pip install -e /content/ucca
!git clone https://github.com/OfirArviv/tupa.git
!cd /content/tupa; git checkout -b  remotes/origin/elmo_weighted_w_special_tokens_to_lstm_mlp
!pip install -e /content/tupa

In [0]:
# download spacy (required for tupa)

!python -m spacy download en_core_web_md

In [0]:
# download elmo (including first download of gdrive)

!wget -O /tmp/gdrive https://docs.google.com/uc?id=0B3X9GlR6EmbnWksyTEtCM0VfaFE&export=download
!chmod +x /tmp/gdrive
!/tmp/gdrive download -r 1Lg2nnocplvi37TJe9rP3Tduh98urUKNx
!mkdir /content/tupa/models
!mv elmo_4_test_sentences_1 /content/tupa/models/

In [0]:
# install our utilities

!pip install -U git+https://github.com/comp-aspects-of-appl-linguistics/relation_extraction_utils.git

In [0]:
# download train.json (whose ID is 1nPpp1zs3_0rkPkCvuz-dSK4YygxO6lw1)

!/tmp/gdrive download 1nPpp1zs3_0rkPkCvuz-dSK4YygxO6lw1

Downloaded 1nPpp1zs3_0rkPkCvuz-dSK4YygxO6lw1 at 50.8 MB/s, total 106.8 MB


In [0]:
# create a sample file (no_relation_sample) with 0.5% of the sentenecs and get the ucca parses for them captured in file 'no_relation_sample_ucca.csv'

!tac_to_csv --input train.json --relation no_relation | awk 'BEGIN {srand()} !/^$/ { if (FNR==1 || rand() <= .005) print $0}'  > no_relation_sample.csv
!parse_ucca --input ./no_relation_sample.csv --output ./no_relation_sample_ucca.csv /content/tupa/models/elmo_4_test_sentences_1/elmo_4_test_sentences_1 

In [0]:
%%bash

cat > triggers << EOF
create
find
launch
found
s
's
build
her
his
co-founder
start
establish
set
founder
set up
form
EOF

In [0]:
%%bash

cat > paths << "EOF"
^C ^A !P >< ^P !A
^E ^A !S >< ^S !A
^C !E !E !P >< ^P !A !C
^E !E >< 
^C ^E !C >< ^C ^A !A
^C !E !R >< ^R !C
^A !P >< ^P ^E !C
^C ^E !C >< ^C ^E !C
^C ^A !P >< ^P ^E !C
^E !C >< ^C ^E !C
^C ^A ^H !H !P >< ^P !A !C
^C ^E !C !C >< ^C ^C ^A !A
^C ^E !E >< ^E ^A !A !A !C
^C ^A !A !E !E >< ^E !C
^A !P >< ^P ^E ^E !C
^A !A !E >< 
^C !E !E !P >< ^P ^E ^E ^A !A ^A !A
^E ^A !P >< ^P !A
^C ^A !P >< ^P !A !E !A !C
^C ^A !P >< ^P !A !C
^C ^E !C >< ^C ^C !C !A !C
^C !E >< 
^A !P >< ^P !A !C
^A !P >< ^P ^A !A
^E ^A !P >< ^P ^H !H !A !C
^C !E !E !P >< ^P !A
^C ^E !E >< ^E ^E !C
^C ^E !C >< ^C ^A !A !C
^C ^E !E >< ^E !C
^E !C >< ^C ^A !A
^C !P >< ^P !A !C
^E ^A !P >< ^P !A !C
^A !P >< ^P ^E ^E ^A !A !C
^E !E !E !P >< ^P !A !C
^C !E !P >< ^P !A !C
^A !P >< ^P ^H !H !A
^E !E !P >< ^P !A !C
^A !P >< ^P !A
^C ^C ^E ^A !P >< ^P !A !E !C !C
^C ^E !E >< ^E ^S !H !A !C
^C !E !E >< ^E ^E ^C !C !E
^C ^A !S !C >< ^C ^S ^E !C
^C ^A !P >< ^P ^H !H !A
^C ^E !C >< ^C ^C ^E !C
^C !E !E !P >< ^P !A !C !C
^E ^A !A !E !P >< ^P ^E ^A ^H !C
^C !E !R >< ^R ^E ^E ^P ^A ^H !H !A !C
^A ^H !H !P >< ^P !A !C
^E ^E !E !E !R >< ^R ^E !C
^C ^E ^A ^H !S !C >< ^C ^S !H !A !C
^A !P >< ^P !A !E !C
^C ^E !E !C !R >< ^R !C
^E ^A !P >< ^P !A !E
^C !H !P >< ^P ^H !H !A !A !E !E !A !E !C
^A !P >< ^P ^E ^A ^E ^A !A
EOF

In [0]:
!echo "total number of lines:" $(( $(wc -l < no_relation_sample_ucca.csv) - 1))
!echo "number of rows with matched triggers (that were not identified) :" $(extract_relations_ucca --include_miss --input no_relation_sample_ucca.csv paths triggers | grep 'considered the following matching triggers' |sed  's/.*considered the following matching triggers: \(.*\)/\1/' | awk '{ sub("\r$", ""); print }'  | sed '/^$/d' | wc -l )
!echo "number of matches:" $(( $(extract_relations_ucca --input no_relation_sample_ucca.csv paths triggers | wc -l) - 1 ))
!echo "number of matches filtered by NER:" $(( $(extract_relations_ucca --input no_relation_sample_ucca.csv paths triggers | append_ner | filter_relations --entity-types ORG PERSON | wc -l) - 1 ))

total number of lines: 250
number of rows with matched triggers (that were not identified) : 115
number of matches: 6
number of matches filtered by NER: 0
